# **Guia Prático SQL**

Bancos de dados geralmente costumam utilizar data no formato: **AAAA-MM-DD (ano, mês e dia)**.

Separadores de decimais são **"."** ao invés de **","**.

Ordem de inserção dos comandos: **Execução (Join se necessário) - Filtro - Agregação - Ordenação - Limite**

Sendo:

In [ ]:
SELECT
    P.Title AS Produto,
    COUNT(O.Id) AS Qtd_Pedidos,
    SUM(O.Total) AS Valor_Total_Vendido
FROM Orders O
#Mostra o nome do produto, a quantidade de pedidos e a soma total das vendas.

INNER JOIN Products P ON O.Product_ID = P.Id
#Junta Orders com Products para acessar o nome do produto.

WHERE O.Discount > 0 AND O.Created_At >= '2023-01-01'
#Filtra os pedidos com desconto (Discount > 0) e somente os a partir de 2023-01-01.

GROUP BY P.Title
#Agrupa os resultados por produto.

HAVING SUM(O.Total) > 1000
#Mantém apenas os produtos com mais de R$1000 em vendas totais (Outro filtro se necessário).

ORDER BY Valor_Total_Vendido DESC
#Ordena os resultados pelos produtos mais vendidos (em valor).

LIMIT 5;
#Exibe somente os 5 produtos mais vendidos.

Desmembrando cada etapa abaixo:

## **Execução**

In [ ]:
SELECT
    Created_At AS Data_Criacao,
    Id AS CodPro,
    Title AS Titulo,
    Category AS Categoria,
    Price AS Preço,
    Rating AS Avaliacao
FROM Products;

### **Operações Aritméticas (resultados em colunas novas)**

In [ ]:
SELECT
    Id AS ID_Pedido,
    User_ID AS Cliente,
    Product_ID AS ID_Produto,
    Subtotal,
    Tax,
    Total,
    Discount,
    Subtotal + Tax AS Soma1,
    Subtotal + Tax - Discount AS Soma2,
    Tax / Subtotal AS Taxa
FROM Orders;

### **Ordenação e Limite (inserindo um limite de linhas para a demonstração das informações)**

In [ ]:
SELECT
    Product_ID AS ID_Produto,
    Subtotal,
    Tax,
    Subtotal + Tax AS Valor_Total,
    Discount,
    Discount / (Subtotal + Tax) * 100 AS Percent_Disc,
    Total
FROM Orders
ORDER BY Percent_Disc DESC
LIMIT 10;

Exemplos de diversos operadores: https://dev.mysql.com/doc/refman/8.4/en/arithmetic-functions.html

Funções numéricas e operadores: https://dev.mysql.com/doc/refman/8.4/en/numeric-functions.html

## **Join - Junção entre Tabelas**


Devem ser aplicados na etapa de execução, logo após a descrição do SELECT

Os Joins (conexões) são possíveis somente entre tabelas que possuem chaves que se comunicam, ou seja, possuem uma chave em comum para ligação (Primary_Keys e Foreign_Keys)


Sendo os principais tipos:

* Left Join - Traz tudo da tabela primária e somente a correspondencia da tabela secundária - Tipo PROCV

* Right Join - Reverte a primária, trazendo a correspondencia da tabela secundária

* Inner Join - Traz somente o que se corresponde nas 2 tabelas

* Full Join - Traz tudo das 2 tabelas, mesmo que não hajam correspondencias

* Cross Join - Repete todas as linhas da Tabela 1 para cada linha da tabela 2

Expressão geral:



In [ ]:
SELECT select_expression FROM table_1

JOIN table_2 ON table_1.key = table_2.key

Neste exemplo:

table_2 #Tabela que queremos AS informações
table_1.key #table_1.key Corresponde a coluna da tabela_1 que contém a chave primária (ponto de conexão)
table_2.key #table_2.key Corresponde a coluna da tabela_2 que contém a chave estrangeira (ponto de conexão)

Aos descrevermos as colunas que queremos trazer no SELECT, podemos trazer as colunas da tabela conectada, por ex:

In [ ]:
  SELECT
       Products.Id AS Product_ID, #Onde os nomes coincidem
       Title,
       Category,
       Price,
       Reviews.Id AS Review_ID,
       Product_ID,
       Reviewer,
       Body
  FROM Products

  LEFT JOIN Reviews ON Products.ID = Reviews.Product_ID

  WHERE Reviews.Id IS null

Colunas com o mesmo nome pode conflitar, o ideal é renomear para um título mais amigável

### Exemplo **LEFT JOIN**

In [ ]:
#Traz tudo da tabela primária e somente a correspondencia da tabela secundária - Tipo PROCV

SELECT
    Users.Id AS ID_Cliente,
    Users.Name AS Nome,
    Orders.Id AS Pedido,
    Orders.Total
FROM Users
LEFT JOIN Orders ON Users.Id = Orders.User_ID;

### Exemplo **RIGHT JOIN**

In [ ]:
#Reverte a primária, trazendo a correspondencia da tabela secundária

SELECT
    Orders.Id AS Pedido,
    Orders.Total,
    Users.Name AS Cliente
FROM Orders
RIGHT JOIN Users ON Orders.User_ID = Users.Id;

### Exemplo **INNER JOIN**

In [ ]:
#Traz somente o que se corresponde nas 2 tabelas

SELECT
    Orders.Id AS ID_Pedido,
    Users.Name AS Cliente,
    Products.Title AS Produto,
    Orders.Total
FROM Orders

INNER JOIN Users ON Orders.User_ID = Users.Id
#Conecta a tabela de Pedidos com a de Usuários, através da chava User_ID

INNER JOIN Products ON Orders.Product_ID = Products.Id;
#Conecta a tabela de Pedidos com a de Produtos, através da chava Products_ID

### Exemplo **FULL OUTER JOIN**

In [ ]:
#Traz tudo das 2 tabelas, mesmo que não hajam correspondencias

SELECT * FROM Users
FULL JOIN Orders ON Users.Id = Orders.User_ID


### Exemplo **CROSS JOIN**

In [ ]:
#Repete todas as linhas da Tabela 1 para cada linha da tabela 2

SELECT * FROM Users
CROSS JOIN Orders

### Aplicando agregadores:

In [ ]:
    SELECT
        COUNT(DISTINCT Products.ID) AS Contagem_DistID,
        Category
    FROM Reviews
    LEFT JOIN Products ON Reviews.Product_ID = Products.ID
    WHERE Category is NOT null
    GROUP BY Category
    ORDER BY Contagem_DistID desc


###**Joins subsequentes - Linkando multiplas tabelas:**

In [ ]:
 SELECT select_expression FROM table_1
    JOIN table_2 ON table_1.key = table_2.key
    JOIN table_3 ON table_2.key = table_3.key
    JOIN table_4 ON table_3.key = table_4.key

### Exemplo Prático - **Cálculo da Média de Rating por estado:**

In [ ]:
    SELECT
        State,
        AVG(Rating) AS Media
    FROM Orders #Tabela contendo AS informações de Produtos e User ID

    LEFT JOIN Products ON Orders.Product_Id = Products.ID (Products contém AS info de Rating - Avaliação)
    LEFT JOIN People ON Orders.User_ID = People.ID (People contém AS info de State - Estado)

    GROUP BY State
    ORDER BY Media desc

O ideal é ter uma linha principal de análise e, a partir dela, conectar as informações que precisamos

## **Filtragem com WHERE**

Antes dos Filtros, o SQL não entederá as colunas renomeadas, de modo que pode dar erro ao chama-las. Devemos utilizar o nome original.

### **Aplicação prática do WHERE**

In [ ]:
SELECT *
FROM Users
WHERE Created_At > '2022-01-01';

SELECT *
FROM Products
WHERE Category = 'electronics';

SELECT *
FROM Orders
WHERE Total > 100 AND Discount > 0;

WHERE category = "Gizmo"
#Texto precisa ser exatamente igual

#A coluna não precisa estar visivel na extração para que seja feito o filtro

### **Aplicando o BETWEEN (Entre)**

In [ ]:
WHERE Month(Dia_Pedido) BETWEEN 1 and 2

WHERE ID >= 4 and ID BETWEEN "10" and "20"

WHERE ID = 4 or ID BETWEEN 1 and 20

WHERE rating != 4 and ID BETWEEN 1 and 20 and Created_At BETWEEN "2019-02-7" and "2019-04-12"
#Aplicando o termo mais de uma vez

### **Principais Operadores Lógicos e Comparativos**



Podemos aplicar o E ou OU:
* and / or - "E" e "OU"
* in() - Nas determinadas condições
* Not - Negação, o Contrário

Podemos aplicar a negação:
* Not In()
* Not Like

Principais Operadores de Comparação:
* "=" (igual)
* "!=" ou "<>" (diferente)
* ">" (maior) ou "<" (menor)
* ">=" (maior ou igual) ou "<=" (menor ou igual)

Mais operadores podem ser encontrados [aqui](https://dev.mysql.com/doc/refman/8.4/en/non-typed-operators.html)



### **Parentesis nos filtros (hierarquia de ação)**

Sem os parênteses, o SQL pode interpretar as condições de forma diferente do que você espera, pois a precedência padrão é:

* AND é avaliado antes de OR.

Exemplo com **Parentesis**:

In [ ]:
SELECT *
FROM Products
WHERE Category = 'electronics'
  AND (Price < 100 OR Rating > 4.5);

#Seleciona produtos da categoria "electronics" e que tenham o preço menor que 100 ou avaliação maior que 4.5.
    #Category = 'electronics' E (Price < 100 OU Rating > 4.5)

Exemplo sem **Parentesis**:

In [ ]:
SELECT *
FROM Products
WHERE Category = 'electronics'
  AND Price < 100 OR Rating > 4.5;

#Interpretação do SQL: (Category = 'electronics' E Price < 100) OU Rating > 4.5
    #Ou seja, o SQL pode retornar produtos de qualquer categoria com avaliação alta, mesmo que não sejam eletrônicos.

Outro exemplo mais prático:

In [ ]:
SELECT *
FROM Orders
WHERE (Status = 'Pago' OR Status = 'Enviado')
  AND Total > 500;

#Seleciona pedidos que já foram pagos ou enviados, e que tenham um valor total acima de 500

### **Cláusula LIKE (Similaridade)**

In [ ]:
SELECT *
FROM Products
WHERE Title LIKE '%phone%';

Combinando com o "%" entre as palavras podemos ser mais específicos, por ex:

In [ ]:
SELECT *
FROM Products
WHERE Category NOT IN ("Gizmo", "Widget")

AND title LIKE "Plastic%"
#Que contenha a palvra Plastic no começo

AND title LIKE "%Plastic%"
#Que contenha a palvra Plastic no meio

AND title LIKE "%Plastic"
#Que contenha a palvra Plastic no fim

AND title LIKE "%Plastic%Watch" -
#Combinando palavras

### **WHERE em Agregações**

Quando precisamos filtrar uma base com valores agregados (como SUM, AVG, COUNT), é necessário usar o WHERE através de uma CTE, Subquery ou uma cláusula HAVING.



 Exemplo onde o WHERE **não funciona** com agregação:

In [ ]:
#Isto dá erro ou comportamento incorreto

SELECT User_ID, SUM(Total) AS Total_Compras
FROM Orders
WHERE SUM(Total) > 500 #Não é permitido usar funções agregadas no WHERE
GROUP BY User_ID;

Aplicando **CTE** (quando queremos usar os resultados intermediários)

In [ ]:
WITH ComprasPorCliente AS
 (SELECT User_ID, SUM(Total) AS Total_Compras
  FROM Orders
  GROUP BY User_ID)

SELECT *
FROM ComprasPorCliente
WHERE Total_Compras > 500;

Aplicando em **Subquery**:

In [ ]:
SELECT *
FROM (
    SELECT User_ID, SUM(Total) AS Total_Compras
    FROM Orders
    GROUP BY User_ID
) AS Compras
WHERE Total_Compras > 500;

Aplicando o **HAVING** (quando há agregações):

In [ ]:
SELECT User_ID, SUM(Total) AS Total_Compras
FROM Orders
GROUP BY User_ID
HAVING SUM(Total) > 500;

## Agregações

### **Funções Agregadas**

In [ ]:
SELECT COUNT(*) FROM Users;
SELECT COUNT(*) FROM Orders WHERE Discount > 0;

SELECT MAX(Price) FROM Products;
SELECT MIN(Price) FROM Products;

SELECT AVG(Rating) FROM Products;
SELECT SUM(Total) FROM Orders;

SELECT
   Category AS Categoria,
   COUNT(*) AS "Qtdade de Produtos"
FROM Products
   GROUP BY Category
   ORDER BY "Qtdade de Produtos"

Importante trazer a coluna que se deseja fazer a agregação na etapa de EXECUÇÃO para que apareça o resumo

É possível fazer multiplas agregações, como por ex:

In [ ]:
    SELECT
        State,
        Source,
        COUNT(*) AS Qtde
    FROM People
    WHERE ID <= 100
    GROUP BY State, source
    ORDER BY State, source

### **Group by**

In [ ]:
SELECT Category, COUNT(*) AS Qtd_Produtos
FROM Products
GROUP BY Category;

SELECT User_ID, SUM(Total) AS Total_Compras
FROM Orders
GROUP BY User_ID;

As principais funções de agregação são:
   * AVG(): Retorna a média dos valores.
   * SUM(): Retorna a soma dos valores.
   * MAX(): Retorna o maior valor.
   * MIN(): Retorna o menor valor.
   * COUNT(): Retorna a quantidade de linhas.
   * COUNT(DISTINCT): Retorna a quantidade de valores distintos.
   * STD(): Retorna o desvio padrão dos valores.

As demais podem ser vistas [aqui](https://dev.mysql.com/doc/refman/8.4/en/aggregate-functions.html)

### **Podemos fazer diversas agregações de uma só vez, por ex:**

In [ ]:
    SELECT
        Category AS Categoria,
        MIN (Rating) AS Nota_Min,
        MAX (Rating) AS Nota_Max,
        AVG (Rating) AS Nota_Med,
        stddev(Rating) AS Desv_Pad
    FROM Products
    GROUP BY Categoria
    ORDER BY Nota_Max

### **Distinct:**

In [ ]:
    SELECT
        DISTINCT State,
        Source
    FROM People
    ORDER BY State
    #Seleciona somente os valores distintos das colunas citadas

In [ ]:
SELECT DISTINCT Category
FROM Products;

É possível aplicar os agregadores e turbinar a conta, trazendo os resultados distintos de uma coluna.
Dependendo da fonte, pode não ser possível aplicar o Distinct em mais de uma coluna

### **HAVING (Filtra após o agrupamento)**

In [ ]:
SELECT User_ID, SUM(Total) AS Total_Compras
FROM Orders
GROUP BY User_ID
HAVING Total_Compras > 500;

#Conforme citado anteriormente, o WHERE pode não funcionar em determinadas ocasiões


## **Ordenação com ORDER BY**

In [ ]:
SELECT *
FROM Products
ORDER BY Rating DESC;

Order by Created_At asc (asc - ascending, desc - descending)

Para ordenar, podemos inserir o nome atual da coluna ou o nome antigo mesmo


### **ORDER BY com GROUP BY**

In [ ]:
SELECT User_ID, COUNT(*) AS Qtd_Pedidos
FROM Orders
GROUP BY User_ID
ORDER BY Qtd_Pedidos DESC;

## **Limite de Resultados com LIMIT**

In [ ]:
SELECT *
FROM Products
ORDER BY Rating DESC
LIMIT 5;

Limita o número de linhas do material


## **Declarando Variáveis - Exemplos de Filtros**

In [ ]:
{{Input_Variavel}} #Abre um campo onde o usuário pode inserir a variável que ele deseja

Pode ser extremamente útil se usarmos com o parametro de filtro (WHERE):

In [ ]:
    SELECT
        *
    FROM Products
    WHERE
        ID = {{input_ID}}
    [[ OR Price >= {{Input_Preco}}
     OR lower(Category) = lower({{Input_Categoria}}) #é possível moldar o texto do input (SQL exige Texto Exato)
     OR Created_At >= {{Input_Data}}
     OR Created_At BETWEEN {{data_ini}} AND {{data_fim}} ]]

[[]] - Colchetes delimitam se o filtro será obrigatório ou não

In [ ]:
#Filtro único
    [[where ID = {{input_ID}}]]

#Filtro multiplo
      where ID = {{input_ID}}          - obrigatório
    [[ and Price >= {{input_Preço}}]]  - não obrigatório

Criando uma caixinha de seleção de valores:

In [ ]:
    SELECT
        *
    FROM Products
    WHERE true - ao inserir essa chave, todos os filtros serão opcionais. Ao não inserir nada, trará a tabela completa.
    [[AND {{input_id}}]]
    [[AND {{input_Categoria}}]]
    [[AND {{input_Data}}]]
    [[AND {{input_Preco}}]]

 Disclaimer Importante:

 A maioria dos interpretadores SQL puros (como SQLite, PostgreSQL, MySQL, etc.) não permite declarar ou usar variáveis diretamente em queries simples.
O uso de variáveis depende do ambiente ou da extensão da linguagem SQL usada, podendo variar bastante a sua delimitação entre eles.

## **Subqueries (Consultas Aninhadas)**

In [ ]:
SELECT *
FROM Orders
WHERE Total > (
    SELECT AVG(Total)
    FROM Orders
);

SELECT Name
FROM Users
WHERE Id IN (
    SELECT User_ID
    FROM Orders
    WHERE Discount > 0
);

## **CTE - Common Table Expression**

Este conceito permite a criação de tabelas virtuais para facilitar as consultas, dentro de tabelas específicas.

In [ ]:
WITH PedidosPorCliente AS (
    SELECT User_ID, SUM(Total) AS Total_Compras
    FROM Orders
    GROUP BY User_ID
)
SELECT
    Users.Name AS Cliente,
    PedidosPorCliente.Total_Compras
FROM PedidosPorCliente
INNER JOIN Users ON PedidosPorCliente.User_ID = Users.Id
WHERE PedidosPorCliente.Total_Compras > 500;

Selecionando todos os pedidos de 2023

In [ ]:
WITH Pedidos2023 AS (
    SELECT
        Id, USER_ID, PRODUCT_ID, SUBTOTAL, cast(CREATED_AT AS DATE) AS Dia
    FROM ORDERS
    WHERE
        year(CREATED_AT) = 2023
    ORDER BY Dia
)

SELECT * FROM Pedidos2023
WHERE month(Dia) = 1

### **DETALHE IMPORTANTE !!!**

No MySQL, uma CTE criada com WITH precisa ser seguida por uma consulta principal que a utilize imediatamente;
do contrário, ele não processa a tabela virtual e gera erro.

Conforme haviamos citado anteriormente, a CTE permite que façamos filtros mais específicos onde antes não podiamos aplicar o WHERE

In [ ]:
WITH FaturamentoDia2023 AS
(
    SELECT
        SUM(SUBTOTAL) AS Faturamento,
        cast(CREATED_AT AS DATE) AS Dia
    FROM ORDERS
    WHERE
        year(CREATED_AT) = 2023
        GROUP BY dia
    ORDER BY dia
)

SELECT * FROM FaturamentoDia2023
WHERE Faturamento >= 1000

Na query acima, não seria possível filtrar o Faturamento sem a tabela virtual, por ser uma coluna agregada daria ERRO.

CTE são visões temporárias, para mantermos as consultas viáveis por mais tempo, temos algumas alternativas:

### **VIEWS**

Uma view é uma tabela virtual salva no banco de dados e pode ser reutilizada em várias consultas.
É como uma CTE, mas ela permanece disponível até que seja explicitamente deletada. Ex:

In [ ]:
    CREATE VIEW Pedidos2023 AS
    SELECT Id, USER_ID, PRODUCT_ID, SUBTOTAL, CAST(CREATED_AT AS DATE) AS Dia
    FROM ORDERS
    WHERE YEAR(CREATED_AT) = 2023;


### **TEMPORARY TABLES**

Elas existem apenas durante a sessão atual do banco de dados e são excluídas automaticamente quando a sessão termina.
Ex:

In [ ]:
    CREATE TEMPORARY TABLE Pedidos2023 AS
    SELECT Id, USER_ID, PRODUCT_ID, SUBTOTAL, CAST(CREATED_AT AS DATE) AS Dia
    FROM ORDERS
    WHERE YEAR(CREATED_AT) = 2023;


### **NOVA TABELA**

Ou até mesmo criar uma nova tabela, caso seja necessário preservar as informações por mais tempo:

In [ ]:
    CREATE TABLE Pedidos2023 AS
    SELECT Id, USER_ID, PRODUCT_ID, SUBTOTAL, CAST(CREATED_AT AS DATE) AS Dia
    FROM ORDERS
    WHERE YEAR(CREATED_AT) = 2023;

## Neste contexto também podemos fazer:

### **Inserção de Dados**

In [ ]:
INSERT INTO Vendas (Produto, Quantidade, Valor, Data)
VALUES ('Mouse', 10, 25.5, '2023-06-01');

### **Atualização de Registros**

In [ ]:
UPDATE Vendas
SET Valor = 30
WHERE Produto = 'Mouse';

### **Exclusão de Registros**

In [ ]:
DELETE FROM Vendas
WHERE Quantidade = 0;

Quando a tabela não for mais necessária:

### **DROP TABLE**

In [ ]:
DROP TABLE Vendas;

## ⚠️ CUIDADOS:

Todos os dados da tabela serão perdidos permanentemente.

A estrutura da tabela também é removida — ou seja, não existe mais a tabela Vendas após a execução.

Não pode ser desfeito (exceto se você tiver backup ou transações com rollback).

Certifique-se de que nenhuma outra tabela depende da que será eliminada (chaves estrangeiras).

Verifique se você não precisa mais dos dados.

✅ Quando usar:

Quando você não precisa mais da tabela nem dos dados.

Durante processos de recriação de estrutura (ex: testes, desenvolvimento).

## **Juntando as CTEs com Joins**

In [ ]:
#Selecionando todos os pedidos feitos em 2023 junto com os clientes criados.

With Pedidos2023 as
(
    select
        cast(CREATED_AT as date) as Dia_Pedido,
        Count(ID) as Pedidos_Feitos
    from ORDERS
    where
        year(CREATED_AT) = 2023
    group by Dia_Pedido
    order by Dia_Pedido
),

Clientes2023 as
(
    Select
        cast(CREATED_AT as date) as Dia_Cliente,
        Count(ID) as Clientes_Cadastrados
    From PEOPLE
    where
        year(CREATED_AT) = 2023
    Group by Dia_Cliente
    Order by Dia_Cliente
)

#Ao inserir a vírgula, podemos criar várias **CTEs dentro de um mesmo WITH**

#Ao fim das CTEs, sempre será necessário criar um SELECT das informações.

Select
    Dia_Pedido as Dia,
    Pedidos_Feitos as Pedidos_Feitos,
    ifnull(Clientes_Cadastrados,0) as Clientes_Cadastrados
from Pedidos2023

left join Clientes2023 on Clientes2023.Dia_Cliente = Pedidos2023.Dia_Pedido

#Join aplicado entre as CTEs com as Chaves comuns entre as 2

where Month(Dia_Pedido) Between 1 and 2
Group by Dia


##**Juntando CTEs com UNION (tabela sob tabela)**

Neste caso, colocaremos uma consulta sobre a outra, de modo que criará uma consulta única com duas informações
Para este caso é importante que as tabelas possuam um mesmo padrão de análise para que não hajam desencontros

Agrupando dados de Filiais:

In [ ]:
#Selecionando cada filial dentro de uma CTE e especificando o período e modo de análise

With FilialNE as
(
    select
        *
    from PEOPLE
    where
        STATE = 'NE' and year(CREATED_AT) = 2023
),

FilialMN as
(
    select
        *
    from PEOPLE
    where
        STATE = 'MN' and year(CREATED_AT) = 2023
),

#Abrindo uma 3ª CTE agrupando as 2 anteriores

Filiais2023 as
(
    select * from FilialNE
    union
    select * from FilialMN
)

#Dessa forma é possível tratar as 2 de uma vez só

Select
    State,
    Count(ID)
from Filiais2023
group by STATE

## **CASE WHEN** (Aplicação do IF)

É uma estrutura condicional (parecida com if em outras linguagens) que permite gerar valores diferentes com base em condições.

In [ ]:
SELECT
    Id,
    Total,
    CASE
        WHEN Total > 1000 THEN 'Alto'
        WHEN Total BETWEEN 500 AND 1000 THEN 'Médio'
        ELSE 'Baixo'
    END AS Faixa_Valor
FROM Orders;

Deste modo, a tabela criará uma coluna com base nas classificações:

* Se o valor total for maior que 1000, a linha receberá 'Alto' na coluna Faixa_Valor.

* Se estiver entre 500 e 1000 (inclusive), receberá 'Médio'.

* Se for menor que 500, receberá 'Baixo'.

Permite a criação de categorias específicas para divisão dos dados, entre outros usos


## **Funções Úteis**

Ppodemos aplicar algumas funções que facilitem a extração das informações das tabelas:

In [ ]:
#Funções de texto:

  select
      Id,
      Title,
      Category,
      Lower(Title) as "CaixaBaixa",
      Upper(Title) as "CaixaAlta",
      Concat(Title,"|",Category) as "ConcatenandoTexto",
      Replace(Title,' ', '') as "SubstituindoValores"
  from Products

In [ ]:
#Funções de Data:

  select
      ID,
      Title,
      Created_At,
      Cast(Created_At as date) as Data, - função Cast muda a tipagem de dado da coluna
      Cast(Created_At as datetime) as DataHora,
      Day(Created_At) as DiadaData,
      Dayofweek(Created_At) as DiadaSemana,
      DayofYear(Created_At) as DiadaSemana,
      Month(Created_At) as Mes,
      Year(Created_At) as Ano,
      Now() as DiaAtual,
      Cast(concat("2024", "/","10","/","29") as date) as CriandoData,
      DateDiff(NOW(),Created_At) as DiferençaData
   from Products

In [ ]:
#Funções de número:

  select
      ID,
      subtotal,
      Discount,
      Ceiling(subtotal),
      Floor(subtotal),
      Round(subtotal, 0),
      coalesce(Discount,0),
      ifnull(Discount,subtotal*0.01),
      cast(subtotal as char)
  from Orders

Documentação com diversas funções de data [aqui](https://dev.mysql.com/doc/refman/8.4/en/date-and-time-functions.html#function_date-add)